# Fit Models on Different Samples With Resampling Ensembles

Ensemble learning are methods that combine the predictions from multiple models. It is important in ensemble learning that the ensemble models are good, making different prediction errors. Predictions that are good in different ways can result in a prediction that is both more stable and often better than the predictions of any individual member model. One way to achieve differences between models is to train each model on a different subset of the available training data. Models are trained on different subsets of the training data naturally through resampling methods such as cross-validation and the bootstrap, designed to estimate the average performance of the model generally on unseen data. The models used in this estimation process can be combined in what is referred to as a resampling-based ensemble, such as a cross-validation ensemble or a bootstrap aggregation (or bagging) ensemble. In this tutorial, you will discover how to develop a suite of different resampling-based ensembles for deep learning neural network models. After completing this tutorial, you will know:

* How to estimate model performance using random splits and develop an ensemble from the models.
* How to estimate performance using 10-fold cross-validation and develop a cross-validation ensemble.
* How to estimate performance using the bootstrap and combine models using a bagging ensemble.

## Resampling Ensembles

Combining the predictions from multiple models can result in more stable predictions, and in some cases, predictions that have better performance than any of the contributing models. Effective ensembles require members that disagree. Each member must have the skill (e.g., perform better than random chance), but ideally, perform well in different ways. Technically, we can say that we prefer ensemble members to have low correlation in their predictions or prediction errors.

One approach to encourage differences between ensembles is to use the same learning algorithm on different training datasets. This can be achieved by repeatedly resampling a training dataset that is in turn used to train a new model. Multiple models are fit using slightly different perspectives on the training data and, in turn, make different errors and often more stable and better predictions when combined. We can refer to these methods generally as data resampling ensembles. A benefit of this approach is that resampling methods may be used that do not use all examples in the training dataset. Any examples that are not used to fit the model can be used as a test dataset to estimate the generalization error of the chosen model configuration. There are three popular resampling methods that we could use to create a resampling ensemble; they are:

* **Random Splits**. The dataset is repeatedly sampled with a random split of the data into train and test sets.
* **k-fold Cross-Validation**. The dataset is split into k equally sized folds, k models are trained, and each fold is given an opportunity to be used as the holdout set where the model is trained on all remaining folds.
* **Bootstrap Aggregation**. Random samples are collected with replacement, and examples not included in a given sample are used as the test set.

Perhaps the most widely used resampling ensemble method is bootstrap aggregation, more commonly referred to as bagging. The resampling with replacement allows more difference in the training dataset, biasing the model and, in turn, resulting in more difference between the predictions of the resulting models. Resampling ensemble models make some specific assumptions about your project:

* A robust estimate of model performance on unseen data is required; if not, then a single train/test split can be used.
* There is a potential for a lift in performance using an ensemble of models; if not, then a single model fit on all available data can be used.
* The computational cost of fitting more than one neural network model on a sample of the training dataset is not prohibitive; if not, all resources should be put into fitting a single model.

Neural network models are remarkably flexible, therefore the lift in performance provided by a resampling ensemble is not always possible given that individual models trained on all available data can perform so well. As such, the sweet spot for using a resampling ensemble is the case where there is a requirement for a robust estimate of performance, and multiple models can be fit to calculate the estimate, but there is also a requirement for one (or more) of the models created during the estimate of performance to be used as the final model (e.g., a new final model cannot be fit on all available training data). Now that we are familiar with resampling ensemble methods, we can work through an example of applying each method in turn.

## Resampling Ensembles Case Study

In this section, we will demonstrate how to use the resampling ensemble to reduce the variance of an MLP on a simple multiclass classification problem. This example provides a template for applying the resampling ensemble to your neural network for classification and regression problems.

### Multiclass Classification Problem

### Single Multilayer Perceptron Model

### Random Splits Ensemble

### Cross-Validation Ensemble

### Bagging Ensemble

## Extensions

This section lists some ideas for extending the tutorial that you may wish to explore.

* **Single Model**. Compare the performance of each ensemble to one model trained on all available data.
* **CV Ensemble Size**. Experiment with larger and smaller ensemble sizes for the cross-validation ensemble and compare their performance.
* **Bagging Ensemble Limit**. Increase the number of members in the bagging ensemble to find the point of diminishing returns.

## Summary

In this tutorial, you discovered how to develop a suite of different resampling-based ensembles for deep learning neural network models. Specifically, you learned:

* How to estimate model performance using random splits and develop an ensemble from the models.
* How to estimate performance using 10-fold cross-validation and develop a cross-validation ensemble.
* How to estimate performance using the bootstrap and combine models using a bagging ensemble.